In [1]:
!pip install pyspark
!pip install findspark

!apt-get update -qq
!apt-get install -y openjdk-11-jdk-headless -qq > /dev/null

!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

!tar xf spark-3.4.1-bin-hadoop3.tgz

# Verifica se a pasta python/lib existe
!ls spark-3.4.1-bin-hadoop3/python/lib/

!pip install pandas


You should consider upgrading via the 'C:\Users\samue\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\samue\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
'apt-get' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
O sistema n�o pode encontrar o caminho especificado.
'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
tar: Error opening archive: Failed to open 'spark-3.4.1-bin-hadoop3.tgz'
'ls' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


You should consider upgrading via the 'C:\Users\samue\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
from pyspark.sql import SparkSession
import os
import pandas as pd

os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-17"  

spark = SparkSession.builder \
    .appName("EnemAnalysis") \
    .master("local[*]") \
    .getOrCreate()

print("Spark Session criada com sucesso!")


Spark Session criada com sucesso!


Leitura do arquivo csv com os microdados do enem 2023


In [29]:
df = spark.read.csv("MICRODADOS_ENEM_2023.csv", header=True, inferSchema=True, sep=';').na.drop()

Mudança do parâmetro 2 e 3 para indicar escola pública e privada, respectivamente

In [24]:
from pyspark.sql.functions import when

df_filtrado = df.filter(df.TP_ESCOLA.isin(2,3)).na.drop()
df_filtrado = df_filtrado.withColumn('TP_ESCOLA', 
                       when(df_filtrado.TP_ESCOLA == '2', "Publica")
                       .when(df_filtrado.TP_ESCOLA == '3', "Privada"))



Criação de colunas entre todos os alunos que tiraram mais que 700 nas competencias no Brasil

In [26]:
df_notas = df_filtrado.withColumn('Acima_700_MT',when(df_filtrado.NU_NOTA_MT >= 700, 1).otherwise(0)) \
                      .withColumn('Acima_700_CN',when(df_filtrado.NU_NOTA_CN >= 700, 1).otherwise(0)) \
                      .withColumn('Acima_700_CH',when(df_filtrado.NU_NOTA_CH >= 700, 1).otherwise(0)) \
                      .withColumn('Acima_700_LC',when(df_filtrado.NU_NOTA_LC >= 700, 1).otherwise(0)) \
                      .withColumn('Acima_700_REDACAO',when(df_filtrado.NU_NOTA_REDACAO >= 700, 1).otherwise(0))

In [27]:
from pyspark.sql.functions import sum as _sum, count

df_resultado = df_notas.groupBy('TP_ESCOLA').agg(count('*').alias('total'),
                                _sum('Acima_700_MT').alias('total_acima_700_mt'),
                                _sum('Acima_700_CN').alias('total_acima_700_cn'),
                                _sum('Acima_700_CH').alias('total_acima_700_ch'),
                                _sum('Acima_700_LC').alias('total_acima_700_lc'),
                                _sum('Acima_700_REDACAO').alias('total_acima_700_redacao'),)
df_resultado.toPandas()

,TP_ESCOLA,total,total_acima_700_mt,total_acima_700_cn,total_acima_700_ch,total_acima_700_lc,total_acima_700_redacao
0,Privada,193554,65134,6638,5555,1193,139117
1,Publica,515059,33776,967,1709,331,173451


Analise de proporção por competência entre escolas públicas e privadas no Brasil

In [28]:
from pyspark.sql.functions import col, round

df_resultado = df_resultado \
    .withColumn("proporcao_acima_700_mt", round(col('total_acima_700_mt') / col('total'), 4)) \
    .withColumn("proporcao_acima_700_cn", round(col('total_acima_700_cn') / col('total'), 4)) \
    .withColumn("proporcao_acima_700_ch", round(col('total_acima_700_ch') / col('total'), 4)) \
    .withColumn("proporcao_acima_700_lc", round(col('total_acima_700_lc') / col('total'), 4)) \
    .withColumn("proporcao_acima_700_redacao", round(col('total_acima_700_redacao') / col('total'), 4))


df_resultado.toPandas()

,TP_ESCOLA,total,total_acima_700_mt,total_acima_700_cn,total_acima_700_ch,total_acima_700_lc,total_acima_700_redacao,proporcao_acima_700_mt,proporcao_acima_700_cn,proporcao_acima_700_ch,proporcao_acima_700_lc,proporcao_acima_700_redacao
0,Privada,193554,65134,6638,5555,1193,139117,0.3365,0.0343,0.0287,0.0062,0.7188
1,Publica,515059,33776,967,1709,331,173451,0.0656,0.0019,0.0033,0.0006,0.3368


Análise de proporção por competência entre escolas públicas e privadas na PB

In [17]:
from pyspark.sql.functions import count

df_notas_pb = df_filtrado.filter(df_notas.SG_UF_PROVA == 'PB').na.drop() 

df_notas_pb_tot = df_notas_pb.groupBy( 'TP_ESCOLA') \
  .agg(count('*').alias('total_estudantes')) \
  .orderBy( 'TP_ESCOLA')
 
df_notas_pb_tot.show(5)

+---------+----------------+
|TP_ESCOLA|total_estudantes|
+---------+----------------+
|  Privada|            4210|
|  Publica|           13090|
+---------+----------------+



In [19]:
df_notas_pb = df_notas_pb.withColumn('Acima_700_MT',when(df_notas_pb.NU_NOTA_MT >= 700, 1).otherwise(0)) \
                      .withColumn('Acima_700_CN',when(df_notas_pb.NU_NOTA_CN >= 700, 1).otherwise(0)) \
                      .withColumn('Acima_700_CH',when(df_notas_pb.NU_NOTA_CH >= 700, 1).otherwise(0)) \
                      .withColumn('Acima_700_LC',when(df_notas_pb.NU_NOTA_LC >= 700, 1).otherwise(0)) \
                      .withColumn('Acima_700_REDACAO',when(df_notas_pb.NU_NOTA_REDACAO >= 700, 1).otherwise(0))
              


In [20]:
df_notas_pb = df_notas_pb.groupBy('TP_ESCOLA').agg(
    count('*').alias('total'),
    _sum('Acima_700_MT').alias('total_acima_700_mt'),
    _sum('Acima_700_CN').alias('total_acima_700_cn'),
    _sum('Acima_700_CH').alias('total_acima_700_CH'),
    _sum('Acima_700_LC').alias('total_acima_700_LC'),
    _sum('Acima_700_REDACAO').alias('total_acima_700_redacao')
)

In [ ]:
from pyspark.sql.functions import col, round


df_notas_pb = df_notas_pb \
    .withColumn("proporcao_acima_700_mt", round(col('total_acima_700_mt') / col('total'), 4)) \
    .withColumn("proporcao_acima_700_cn", round(col('total_acima_700_cn') / col('total'), 4)) \
    .withColumn("proporcao_acima_700_ch", round(col('total_acima_700_ch') / col('total'), 4)) \
    .withColumn("proporcao_acima_700_lc", round(col('total_acima_700_lc') / col('total'), 4)) \
    .withColumn("proporcao_acima_700_redacao", round(col('total_acima_700_redacao') / col('total'), 4))

df_notas_pb.toPandas()

,TP_ESCOLA,total,total_acima_700_mt,total_acima_700_cn,total_acima_700_CH,total_acima_700_LC,total_acima_700_redacao,proporcao_acima_700_mt,proporcao_acima_700_cn,proporcao_acima_700_ch,proporcao_acima_700_lc,proporcao_acima_700_redacao
0,Privada,4210,1315,127,135,27,3333,0.3124,0.0302,0.0321,0.0064,0.7917
1,Publica,13090,578,15,24,2,5215,0.0442,0.0011,0.0018,0.0002,0.3984
